In [ ]:
import torch
import torch.nn as nn
import math
import pandas as pd
import string
import numpy as np
from tqdm import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- Vocabulary Class ---
class Vocabulary:
    def __init__(self):
        self.char2idx = {}
        self.idx2char = {}
        self.pad_token = 0
        self.sos_token = 1
        self.eos_token = 2
        self.unk_token = 3
        self._build_vocab()

    def _build_vocab(self):
        special_tokens = ['<PAD>', '<SOS>', '<EOS>', '<UNK>']
        all_chars = list(string.printable)
        self.char2idx = {token: idx for idx, token in enumerate(special_tokens)}
        self.char2idx.update({char: idx+len(special_tokens) for idx, char in enumerate(all_chars)})
        self.idx2char = {idx: char for char, idx in self.char2idx.items()}

    def __len__(self):
        return len(self.char2idx)

    def encode(self, text):
        if isinstance(text, float):  # Handle NaN values
            text = str(text)
        return [self.char2idx.get(char, self.unk_token) for char in text]

    def decode(self, indices):
        return ''.join([self.idx2char.get(idx, '<UNK>') for idx in indices if idx not in {self.pad_token, self.sos_token, self.eos_token}])

# --- Transformer Model ---
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn_probs = torch.softmax(scores, dim=-1)
        return torch.matmul(attn_probs, V)

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        return self.W_o(self.combine_heads(attn_output))

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super().__init__()
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        return self.norm2(x + self.dropout(ff_output))

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        return self.norm3(x + self.dropout(ff_output))

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_pad_mask = (tgt != 0).unsqueeze(1).unsqueeze(2)
        tgt_len = tgt.size(1)
        tgt_sub_mask = torch.tril(torch.ones((tgt_len, tgt_len), device=device)).bool()
        tgt_mask = tgt_pad_mask & tgt_sub_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        return self.fc(dec_output)

# --- Inference Functions ---
def decrypt_text(model, text, vocab, max_length, device):
    model.eval()
    with torch.no_grad():
        encoded = [vocab.sos_token] + vocab.encode(text) + [vocab.eos_token]
        encoded = encoded + [vocab.pad_token] * (max_length - len(encoded))
        encoded = torch.tensor(encoded[:max_length]).unsqueeze(0).to(device)
        target = torch.tensor([[vocab.sos_token]]).to(device)

        for _ in range(max_length - 1):
            output = model(encoded, target)
            next_token = output.argmax(2)[:, -1].item()
            if next_token == vocab.eos_token:
                break
            target = torch.cat([target, torch.tensor([[next_token]]).to(device)], dim=1)

        return vocab.decode(target[0].cpu().numpy())

def process_and_print_results(model, test_file, vocab, max_length, device, num_samples=4000):
    try:
        df = pd.read_excel(test_file) if test_file.endswith('.xlsx') else pd.read_csv(test_file)
    except Exception as e:
        print(f"Error loading test file: {e}")
        return None

    if len(df) > num_samples:
        df = df.sample(n=num_samples, random_state=42)

    results = []
    total_correct = 0
    total_chars_correct = 0
    total_chars = 0

    print("\n=== Starting Inference ===")
    print(f"Processing {len(df)} test cases...\n")

    for idx, row in df.iterrows():
        try:
            input_text = str(row['input'])
            expected = str(row['output'])

            # Decrypt the text
            decrypted = decrypt_text(model, input_text, vocab, max_length, device)
            decrypted_clean = decrypted.replace('<SOS>', '').replace('<EOS>', '').strip()

            # Calculate metrics
            full_match = decrypted_clean == expected
            total_correct += int(full_match)

            min_len = min(len(decrypted_clean), len(expected))
            char_matches = sum(1 for a, b in zip(decrypted_clean[:min_len], expected[:min_len]) if a == b)
            total_chars_correct += char_matches
            total_chars += max(len(decrypted_clean), len(expected))

            # Print detailed results for each case
            print(f"\n--- Case {idx+1} ---")
            print(f"Input:     {input_text}")
            print(f"Decrypted: {decrypted_clean}")
            print(f"Expected:  {expected}")
            print(f"Match:     {'✅' if full_match else '❌'}")
            print(f"Char Accuracy: {char_matches}/{max(len(decrypted_clean), len(expected))} ({(char_matches/max(len(decrypted_clean), len(expected)))*100:.1f}%)")

            results.append({
                'Case #': idx+1,
                'Input': input_text,
                'Decrypted': decrypted_clean,
                'Expected': expected,
                'Full Match': 'Yes' if full_match else 'No',
                'Char Matches': f"{char_matches}/{max(len(decrypted_clean), len(expected))}",
                'Char Accuracy': f"{(char_matches/max(len(decrypted_clean), len(expected)))*100:.2f}%"
            })

        except Exception as e:
            print(f"\nError processing case {idx+1}: {e}")
            continue

    # Create results DataFrame
    results_df = pd.DataFrame(results)

    # Add summary
    full_acc = (total_correct/len(results))*100 if results else 0
    char_acc = (total_chars_correct/total_chars)*100 if total_chars > 0 else 0

    summary = pd.DataFrame([{
        'Case #': 'SUMMARY',
        'Input': f"Total Cases: {len(results)}",
        'Decrypted': f"Full Accuracy: {full_acc:.2f}%",
        'Expected': f"Char Accuracy: {char_acc:.2f}%",
        'Full Match': f"{total_correct}/{len(results)}",
        'Char Matches': f"{total_chars_correct}/{total_chars}",
        'Char Accuracy': f"{char_acc:.2f}%"
    }])

    return pd.concat([results_df, summary], ignore_index=True)

# --- Main Execution ---
if __name__ == "__main__":
    # Configuration - UPDATE THESE PATHS
    MODEL_PATH = '/content/drive/MyDrive/mono_vanilla_one_key.pth'
    TEST_FILE = '/content/Full_testing_mono_one.xlsx'
    OUTPUT_FILE = '/content/drive/MyDrive/inference_mono_one_vanilla.xlsx'
    MAX_LENGTH = 256

    # Initialize vocabulary and model
    vocab = Vocabulary()
    model = Transformer(
        src_vocab_size=len(vocab),
        tgt_vocab_size=len(vocab),
        d_model=512,
        num_heads=8,
        num_layers=6,
        d_ff=1024,
        max_seq_length=MAX_LENGTH,
        dropout=0.12866753791718177
    ).to(device)

    # Load trained model
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        model.eval()
        print("Model loaded successfully")
    except Exception as e:
        print(f"Error loading model: {e}")
        exit()

    # Process test cases and print all results
    results = process_and_print_results(model, TEST_FILE, vocab, MAX_LENGTH, device)

    # Save to Excel
    if results is not None:
        try:
            results.to_excel(OUTPUT_FILE, index=False)
            print(f"\nFull results saved to {OUTPUT_FILE}")

            # Print final summary
            print("\n=== Final Summary ===")
            print(f"Full-match accuracy: {results.iloc[-1]['Full Match']}")
            print(f"Character-level accuracy: {results.iloc[-1]['Char Matches']} ({results.iloc[-1]['Char Accuracy']})")
        except Exception as e:
            print(f"\nError saving results: {e}")
    else:
        print("\nNo results were generated")

Streaming output truncated to the last 5000 lines.
Expected:  SHE SPEAKS PROPER BERBER.
Match:     ✅
Char Accuracy: 25/25 (100.0%)

--- Case 3288 ---
Input:     EXDE BT D IOHO OUKPTO AWH BGLOROTT.
Decrypted: THAT IS A MERE EXCUSE FOR IDLENESS.
Expected:  THAT IS A MERE EXCUSE FOR IDLENESS.
Match:     ✅
Char Accuracy: 35/35 (100.0%)

--- Case 3289 ---
Input:     TDIB MDEKXOG XBT QDHORET ZW EXHWPZX D GBNWHKO.
Decrypted: SAMI WATCHED HIS PARENTS GO THROUGH A DIVORCE.
Expected:  SAMI WATCHED HIS PARENTS GO THROUGH A DIVORCE.
Match:     ✅
Char Accuracy: 46/46 (100.0%)

--- Case 3290 ---
Input:     B DI MXBEO.
Decrypted: I AM WHITE.
Expected:  I AM WHITE.
Match:     ✅
Char Accuracy: 11/11 (100.0%)

--- Case 3291 ---
Input:     B LBVO EXO ROM KDHQOE.
Decrypted: I LIKE THE NEW CARPET.
Expected:  I LIKE THE NEW CARPET.
Match:     ✅
Char Accuracy: 22/22 (100.0%)

--- Case 3292 ---
Input:     VDJFLBO BT D TQOKBDL QLDKO.
Decrypted: KABYLIE IS A SPECIAL PLACE.
Expected:  KABYLIE IS A SPECIAL PLACE.